## Segmenting and Clustering Neighborhoods in Toronto

### Scraping Wikipedia page and transform the data using a pandas dataframe

Import all the required library

In [1]:
import pandas as pd
import geocoder

Scrap data from Wikipedia page as per the URL provided. Check for the list length

In [2]:
# Use pandas to import the data
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Find the list length
len(wiki)

3

First element in the list contains the table with data

In [3]:
# Extract the Table with the data from the list
Postal_Code = wiki[0]
Postal_Code.head()

,0,1,2
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Need to assign the first row as Column Name

In [4]:
header = Postal_Code.iloc[0]
Postal_Code = Postal_Code[1:]
Postal_Code.columns = header
Postal_Code.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Rename Postal code as per the requirement description

In [5]:
Postal_Code = Postal_Code.rename(columns={'Postal code': 'PostalCode'})
Postal_Code.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Only process the cells that have an assigned Borough, ignore records with **Not assigned** values

In [7]:
# Ignore cells with a borough that is "Not assigned"
Postal_Code = Postal_Code.drop(Postal_Code[Postal_Code['Borough'] == 'Not assigned'].index)
Postal_Code = Postal_Code.reset_index(drop = True)
Postal_Code.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one Neighborhood can exist for a Postal Code. Such rows will be combined into one row with the neighborhoods separated with a **comma** 


In [8]:
# Check for more than one neighborhood in one postal code area
duplicateRowsDF = Postal_Code[Postal_Code.duplicated()] 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


#### The data seems to already have the above condition handled but with a /(slash) as the separator. Replaced /(slash) with comma

In [9]:
# More than one neighborhood for a postal code area is separated by '/' in the original data.As per requirement, replaced '/' with ','
Postal_Code['Neighborhood'] = Postal_Code['Neighborhood'].str.replace(" / ",", ")
Postal_Code.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Rename Postal code column to match with the output in the problem description

If a cell has a Borough but a **Not assigned** neighborhood, then the Neighborhood will be the same as the Borough

In [10]:
# If Borough has a valid value but a "Not assigned" in Neighborhood then Neighborhood needs to be replaced with the value of Borough
Postal_Code.loc[Postal_Code['Neighborhood'] == 'Not assigned','Neighborhood'] = Postal_Code.loc[Postal_Code['Neighborhood'] == 'Not assigned','Borough']
Postal_Code.loc[Postal_Code['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


Print number of rows of the dataframe

In [12]:
# Find the dataframe dimension
Postal_Code.shape

(103, 3)